In [1]:
import matplotlib
matplotlib.use('TkAgg')  # Use TkAgg backend for matplotlib
from matplotlib import pyplot as plt
from PIL import Image
import torch
from transformers import GLPNImageProcessor, GLPNForDepthEstimation



In [2]:
feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")



preprocessor_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/245M [00:00<?, ?B/s]

In [3]:
image = Image.open("../data/food.jpg")
new_height = 480 if image.height > 480 else image.height
new_height -=(new_height % 32)  # Ensure height is a multiple of 32
new_width =int(new_height * image.width / image.height)
diff = new_width % 32

new_width = new_width - diff  if diff < 16 else new_width + 32 - diff
new_size = (new_width, new_height)
image = image.resize(new_size)

In [ ]:
inputs = feature_extractor(images=image, return_tensors="pt")


In [5]:
with torch.no_grad():
    outputs = model(**inputs)
    
    predicted_depth = outputs.predicted_depth

In [6]:
pads =15
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pads:-pads, pads:-pads]
image = image.crop((pads, pads, image.width - pads, image.height - pads))


In [8]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image)
ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
ax[1].imshow(output, cmap='plasma')
ax[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.tight_layout()
plt.pause(5)

In [9]:
import numpy as np
import open3d as o3d



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [12]:
width, height = image.size

depth_image =(output *255 /np.max(output)).astype(np.uint8)
image = np.array(image)

depth_o3d = o3d.geometry.Image(depth_image)
image_o3d = o3d.geometry.Image(image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth
(image_o3d, depth_o3d,convert_rgb_to_intensity=False)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (1684538789.py, line 9)